In [20]:
def LinAprox(x0, y0, x1, y1, X):
    a = (y1-y0) / (x1-x0)
    b = y0 - a * x0
    W = a * X + b
    return W

In [21]:
import numpy as np
import pandas as pd
import math
material = pd.read_excel('Frezowanie rowkow.xlsx',sheet_name='Material')
dfRa = pd.read_excel('qRowkiWpustowe.xlsx',sheet_name='Chropowatość')
dfDims = pd.read_excel('qRowkiWpustowe.xlsx',sheet_name='Wymiary')
dfLen = pd.read_excel('qRowkiWpustowe.xlsx',sheet_name='Długość')

predkosc = pd.read_excel('Frezowanie rowkow.xlsx',sheet_name='Predkosc')
posuw = pd.read_excel('Frezowanie rowkow.xlsx',sheet_name='Posuw')
opor = pd.read_excel('Frezowanie rowkow.xlsx',sheet_name='Opor')

#### każdy rowek będzie miał następujące cechy: 
#L - długość rowka 
#R - szerokość rowka 
#G - głębokość rowka 
#D - średnia powierzchni cylindrycznej wału na której rowek jest wykonany
#Fit - pasowanie określające klasę dokładności wykonania oraz jego położenie (ciasne, luźne)
#Ra – chropowatość powierzchni wewnętrznych rowka
#Mat – materiał z jakiego wykonany jest przedmiot obrabiany. Jest identyfikowany przez oznaczenie np. „St4S”, 
#ale charakteryzują go różne parametry m.in.
#o	Rm – wytrzymałość materiału na rozciąganie [MPa]
#o	HB – twardość materiału wg skali Brinella

#Wykonane obliczenia dotyczą 
#drogi skrawania L
#posuwu p
#posuwu na ząb
#szybkości skrawania V
#prędkości obrotowej
#posuw minutowy
#czas głowny
#moc

df = pd.DataFrame([], columns=('Mat','Rm','HB','Ra','Fit','R','G','D','L','Vc','Vf','n','f','ap','IL','Tg','pz','Amax','ks','Fs','M','Ps','Pe'))
for i in range(10000):
    # wybranie materiału
    Mmin = material['Nr'].min()
    Mmax = material['Nr'].max()
    MID = np.random.randint(Mmin,Mmax)
    MatSelected = material[material['Nr']==MID]
    # nazwa wg PN
    
    MatPN = MatSelected['Mat_PN'].iloc[0]
    Rm = MatSelected['Rm'].iloc[0]
    HB = MatSelected['HB'].iloc[0]
    
    Ramin = dfRa['Nr'].min()
    Ramax = dfRa['Nr'].max() 
    RaID = np.random.randint(Ramin,Ramax)
    Ra_Selected = dfRa[dfRa['Nr']==RaID]
    Ra = Ra_Selected['Ra'].iloc[0]
    Fit = 'N9'
    
    DimMin = dfDims['Nr'].min()
    DimMax = dfDims['Nr'].max()
    DID = np.random.randint(DimMin, DimMax)
    Dim_Selected = dfDims[dfDims['Nr']==DID]
    R = Dim_Selected['b_R'].iloc[0]
    G = Dim_Selected['t1_G'].iloc[0]
    
    Dmin = Dim_Selected['Dmin'].iloc[0]
    Dmax = Dim_Selected['Dmax'].iloc[0]
    D = np.random.randint(Dmin,Dmax)
    
    
    Selected_Lens = dfLen[dfLen['b_R']==R]
    NRmin = Selected_Lens['Nr'].min()
    NRmax = Selected_Lens['Nr'].max()
    NrID = np.random.randint(NRmin,NRmax)
    NR_Selected = Selected_Lens[Selected_Lens['Nr']==NrID]
    L = NR_Selected['L'].iloc[0]
    
    # MC to MatSelected
    MC = material[material['Mat_PN']==MatPN]
    MKod = MatSelected['MatKod'].iloc[0]
    MKod
    SelectedVc = predkosc[predkosc['MatKod'] == MKod]
    SelectedVc
    Vc = SelectedVc['Vc'].iloc[0]
    SelF = posuw[posuw['MatKod'] == MKod]
    SelF2 = SelF[(SelF['Dmin']<=R) & (SelF['Dmax']>R)]
    Vf = SelF2['Posuw'].iloc[0]
    n = 318 * Vc / R
    f = Vf / n
    ap = 0.1 * R
    IL = math.ceil((G + 1.5) / ap)
    Tg = (IL * L) / Vf
    z = 2
    pz = f / z
    
    # Przekrój warstwy skrawanej
    Amax = 2 * pz * math.sqrt(ap / R)
    MProp = MC['Prop'].iloc[0]
    SelOP = opor[opor['MatProp']==MProp]
    SelOP['Roznica'] = SelOP['amax'] - Amax

    for o in range(9):
        SelOP['Roznica'].iloc[o] = math.fabs(SelOP['Roznica'].iloc[o])

    Rmin = SelOP['Roznica'].min()
    IDX = SelOP[SelOP['Roznica']==Rmin]
    HBmin = IDX['HB_min'].iloc[0]
    HBmax = IDX['HB_max'].iloc[0]
    KsMin = IDX['ks_min'].iloc[0]
    KsMax = IDX['ks_max'].iloc[0]
    HB = MC['HB'].iloc[0]
    ks = LinAprox(HBmin, KsMin, HBmax, KsMax, HB)
    Fs = (ks * ap * f) / math.pi
    M = (Fs * D) / 2
    Ps = (M * n) / 9554
    Pe = Ps / 0.6
    
    
    newSeries = pd.Series(data=(MatPN,Rm,HB,Ra,Fit,R,G,D,L,Vc,Vf,n,f,ap,IL,Tg,pz,Amax,ks,Fs,M,Ps,Pe), index=('Mat', 'Rm', 'HB', 'Ra', 'Fit', 'R', 'G', 'D', 'L','Vc','Vf','n','f','ap','IL','Tg','pz','Amax','ks','Fs','M','Ps', 'Pe'))
    df = df.append(newSeries, ignore_index=True)

C:\Users\ssarka\AppData\Local\Temp/ipykernel_21052/614483978.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SelOP['Roznica'] = SelOP['amax'] - Amax
C:\Users\ssarka\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [22]:
df

,Mat,Rm,HB,Ra,Fit,R,G,D,L,Vc,...,ap,IL,Tg,pz,Amax,ks,Fs,M,Ps,Pe
0,MSt5,500,160,0.63,N9,25,9.0,89,220,30,...,2.5,5,13.750000,0.104822,0.066295,3920.000000,653.969997,29.101665,1.162361,1.937268
1,40H,1000,217,5.00,N9,12,5.0,39,36,15,...,1.2,6,4.235294,0.064151,0.040573,6418.571429,314.559492,6.133910,0.255205,0.425342
2,30G2,800,207,6.30,N9,16,6.0,54,90,15,...,1.6,5,8.823529,0.085535,0.054097,6035.000000,525.798284,14.196554,0.442992,0.738320
3,20,420,156,5.00,N9,10,5.0,34,28,30,...,1.0,7,1.960000,0.052411,0.033148,3705.000000,123.620349,2.101546,0.209847,0.349744
4,0H18N9,600,215,1.25,N9,32,11.0,118,125,10,...,3.2,4,23.809524,0.105660,0.066825,5657.142857,1217.697473,71.844151,0.747280,1.245467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,30H,900,187,2.50,N9,3,1.8,9,18,15,...,0.3,11,4.950000,0.012579,0.007955,5992.500000,14.396015,0.064782,0.010781,0.017969
9996,St4S,420,140,3.20,N9,22,9.0,76,70,30,...,2.2,5,4.375000,0.092243,0.058340,2800.000000,361.738833,13.746076,0.623906,1.039843
9997,St3S,380,120,6.30,N9,12,5.0,43,56,30,...,1.2,6,3.360000,0.062893,0.039777,2670.000000,128.284890,2.758125,0.229507,0.382512
9998,30H,900,187,1.25,N9,22,9.0,80,180,15,...,2.2,5,18.750000,0.110692,0.070008,4446.500000,689.345022,27.573801,0.625759,1.042932


In [23]:
df.to_excel('Wygenerowane dane.xlsx')